#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [1]:
#names here Ian Chen

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [3]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"

    def __init__(self, label, features):
        self.label = label  # the classification label of this data point
        self.features = features  # a list of feature values for this data point

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [4]:
def get_data(filename):
    data = []
    # your code goes here
    df = pd.read_csv(filename, header=None)
    features_df = df.iloc[:, :-1]
    labels_df = df.iloc[:, -1]

    for i in range(len(df)):
        features = list(features_df.iloc[i])
        label = labels_df.iloc[i]
        data.append(DataPoint(label, features))

    return data

In [5]:
class TreeNode:
    is_leaf = True  # boolean variable to check if the node is a leaf
    feature_idx = None  # index that identifies the feature
    thresh_val = None  # threshold value that splits the node
    prediction = None  # prediction class (only valid for leaf nodes)
    left_child = None  # left TreeNode (all values < thresh_val)
    right_child = None  # right TreeNode (all values >= thresh_val)

    def printTree(self, level=0):  # for debugging purposes
        if self.is_leaf:
            print("-" * level + "Leaf Node:      predicts " + str(self.prediction))
        else:
            print(
                "-" * level
                + "Internal Node:  splits on feature "
                + str(self.feature_idx)
                + " with threshold "
                + str(self.thresh_val)
            )
            self.left_child.printTree(level + 1)
            self.right_child.printTree(level + 1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [6]:
def make_prediction(tree_root, data_point):
    # your code goes here
    if tree_root is None:
        return None
    current_node = tree_root
    while not current_node.leaf:
        if data_point.feature[current_node.feature_idx] < current_node.thresh_val:
            current_node = current_node.left_child
        else:
            current_node = current_node.right_child

    return current_node.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [7]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    # your code goes here
    # Assuming data is a Python list
    for i in range(len(data)):
        if data[i][feature_idx] < threshold:
            left_split.append(data[i])
        else:
            right_split.append(data[i])
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [8]:
def calc_entropy(data):
    entropy = 0.0
    # your code goes here
    # Assuming data is a Python list
    # Bin the different class labels
    labels = dict()
    for i in range(len(data)):
        label = data[i].label
        if label in labels:
            labels[label] += 1
        else:
            labels[label] = 1
    # Calculate entropy using \sum_{i=1}^{c}-p_{i}\log_{2}p_{i}
    total = sum(labels.values())
    if total != len(data):
        raise ArithmeticError(
            f"Total labels: {total} not equal to length of data: {len(data)}"
        )
    for count in labels.values():
        p = count / total
        entropy += -p * log2(p)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [ ]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    # your code goes here
    thresholds = [dp.features[feature_idx] for dp in data]
    labels = [dp.label for dp in data]
    data = sorted(zip(thresholds, labels), key=lambda x: x[0])
    cases = len(data)
    no_before, yes_before = 0, 0
    previous_entropy = calc_entropy(data)
    lowest_entropy = previous_entropy
    best_thresh = 0
    # TODO: add check for different labels but same threshold
    for i in range(1, len(data)):
        if data[i][1] != data[i - 1][1]:
            before_entropy = calc_entropy(data[:i])
            after_entropy = calc_entropy(data[i:])
            total_entropy = (i / cases) * before_entropy + (
                (cases - i) / cases
            ) * after_entropy
            if total_entropy < lowest_entropy:
                lowest_entropy = total_entropy
                best_thresh = i
    best_info_gain = previous_entropy - lowest_entropy
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [ ]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
    # your code goes here
    best_gain = 0
    for f in range(len(data[0].features)):
        gain, thresh = calc_best_threshold(data, f)
        if gain > best_gain:
            best_feature = f
            best_thresh = thresh
            best_gain = gain
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [ ]:
def create_leaf_node(data):
    # your code goes here
    total_no = sum(dp.label == 0 for dp in data)
    total_yes = len(data) - total_no
    predicted_label = 1 if total_yes >= total_no else 0
    # TODO: check why all TreeNode variables are class variables
    return TreeNode()

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root). [Hint: this is where the recursion happens.]

In [ ]:
def create_decision_tree(data, max_levels):
    # your code goes here
    return None

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [ ]:
def calc_accuracy(tree_root, data):
#     your code goes here
    return 0.0

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [ ]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

# partition data into train_set and test_set
train_set = None
test_set = None

print ('Training set size:', len(train_set))
print ('Test set size    :', len(test_set))

# the timer is just for fun! you will NOT be graded on runtime
start = time.time()

# create the decision tree
tree = create_decision_tree(train_set, 10)

end = time.time()
print ('Time taken:', end - start)

# calculate the accuracy of the tree
accuracy = calc_accuracy(tree, test_set)
print ('The accuracy on the test set is ', str(accuracy * 100.0))
#t.printTree()